In [154]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from PIL import Image
import albumentations as A
import matplotlib.pyplot as plt
import math

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
import zipfile
import shutil
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dogs-vs-cats/test1.zip
/kaggle/input/dogs-vs-cats/train.zip
/kaggle/input/dogs-vs-cats/sampleSubmission.csv
/kaggle/input/cell1-s1/cp.ipynb.tif
/kaggle/input/cells1-particle-positions-with-starting-point/CellS1_particle_positions_with_starting_point.csv


In [155]:
path_to_tif = '/kaggle/input/cell1-s1/cp.ipynb.tif'

image = Image.open(path_to_tif)
print("Image mode:", image.mode)

Image mode: I;16


In [156]:
from PIL import Image
import numpy as np
import pandas as pd

# Load .tif image and normalize frames to [0, 1]
path_to_tif = '/kaggle/input/cell1-s1/cp.ipynb.tif'
image = Image.open(path_to_tif)

frames = []
for i in range(image.n_frames):
    image.seek(i)
    frame_array = np.array(image).astype(np.float32) / 65535.0  # Normalize to [0, 1]
    frames.append(frame_array)

# Get image dimensions (assuming all frames have same size)
img_height, img_width = frames[0].shape

# Load and normalize keypoints
path_to_csv = '/kaggle/input/cells1-particle-positions-with-starting-point/CellS1_particle_positions_with_starting_point.csv'
df = pd.read_csv(path_to_csv)

# Normalize keypoints to [0, 1]
df['x_norm'] = df['centroidGlobalColX'] / img_width
df['y_norm'] = df['centroidGlobalRowY'] / img_height

# Create list of normalized keypoints
keypoints = list(zip(df['x_norm'], df['y_norm']))

In [157]:
IMG_SIZE = img_width if img_width==img_height else -1

In [158]:
transform = A.Compose([
    A.Rotate(limit=40, p=1.0),
    A.HorizontalFlip(p=1.0),
    A.ShiftScaleRotate(shift_limit=0.2, scale_limit=0.2, rotate_limit=0, p=1.0),
    A.RandomBrightnessContrast(p=0.5)
], keypoint_params=A.KeypointParams(format='xy', remove_invisible=False))


In [159]:
def make_batches(images, keypoints, batch_size):
    total = len(images)
    num_batches = math.ceil(total / batch_size)

    batches = []
    for i in range(num_batches):
        start = i * batch_size
        end = start + batch_size
        batch_images = images[start:end]
        batch_keypoints = keypoints[start:end]
        batches.append((batch_images, batch_keypoints))
    return batches


In [160]:
batch_size = 32
augmented_batches = {}

batches = make_batches(images=frames, keypoints=keypoints, batch_size=batch_size)
# Make a |frames| x N set of data
# All identical right now
# Shape: [[array(img), keypoints], ... ]
    #                               ^
    #                            batch Entries
len(batches)

10

In [161]:
for index, (bimg_batch, bkp_batch) in enumerate(batches):
    augmented_images = []
    augmented_keypoints = []

    for bimg, bkp in zip(bimg_batch, bkp_batch):
        # If bkp is a single keypoint (x, y), wrap it
        augmented = transform(image=bimg, keypoints=[bkp])
    

        
        augmented_images.append(np.divide(augmented['image'],IMG_SIZE))
        augmented_keypoints.append(np.divide(augmented['keypoints'], IMG_SIZE))  # Unwrap single keypoint

        

    augmented_batches[index] = (augmented_images, augmented_keypoints)


In [162]:
for index, (images, keypoints) in augmented_batches.items():
    np.savez(f'./augs/batch_{index}.npz', images=np.array(images, dtype=object), keypoints=np.array(keypoints, dtype=object))


In [163]:
import zipfile
import glob

zip_path = "/kaggle/working/Augmentations.zip"
npz_files = glob.glob("/kaggle/working/augs/*.npz")

with zipfile.ZipFile(zip_path, 'w') as zipf:
    for file in npz_files:
        zipf.write(file, arcname=file.split('/')[-1])  # saves only the filename in the archive
